# Moisture Simulation

In this notebook, we simulate moisture dynamics for N > 100 precip events. The resulting time series is written to a .csv, which is used in other notebooks to properly be used to verify our analytical calculations.

In [1]:
# Import general functions
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl 
from random import randrange 
import pandas as pd
from scipy.special import lambertw

mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'

cms = mpl.cm 

In [2]:
import sys
sys.path.append("/data/keeling/a/adammb4/heatwaves-physics/src/")
from getSimluationFluxes import * 
from getAnalyticVerificationFuncs import *
from getSimulate import * 

## Model setup

Here we make an array of model parameters and simluation parameters.

In [3]:
# define model parameters 
C = 4180 # J/K
F = 500 # W / m^2
alpha = 10 # W / K for now, usually between 5 and 25
v_L = 1.25 * 10**(-2) # kg air / m^2 / s
L = 2.5 * 10**6 # J / kg H20
gamma = 3.7 * 10**(-4) # kg H20 / kg air / K
T_D = 5 + 273 # K 
mu = 50 # kg / m^2

param_list = [C, F, alpha, v_L, L, gamma, T_D, mu]

## Full fledged simulation

Here we run the simulation until we've hit a specified amount of precip events.

In [ ]:
# setup lists
time_list = [0]
mois_list = [0]
precip_list = []

# loop quantities
freq_tracker = 1
precip_tracker = 1
N_precip_events = 1000
iter_tracker = 1

# precip dist params
precip_mean = 0.1
precip_scale = 0.5

# freq dist params
omega = 3.5 * 86400

# run simulation until the enough precip events are recorded
while precip_tracker <= N_precip_events:
    if iter_tracker == freq_tracker:
        tmp_precip = np.random.gamma(precip_mean, precip_scale)
        tmp_mois = mois_list[iter_tracker - 1] + getMFlux1D(mois_list[iter_tracker-1], param_list, tmp_precip)
        
        mois_list.append(tmp_mois)
        precip_list.append(tmp_precip)
        
        freq_tracker += int(np.random.poisson(omega))
        precip_tracker += 1
        #print(precip_tracker)
    
    else:
        tmp_mois = mois_list[iter_tracker - 1] + getMFlux1D(mois_list[iter_tracker-1], param_list, 0)
        
        mois_list.append(tmp_mois)
    time_list.append(iter_tracker)
    iter_tracker += 1

# make pandas dataframe to write data (with mismatching size) to csv for further analysis
d = dict( time = np.array(time_list), moisture = np.array(mois_list), precip = np.array(precip_list))
df = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in d.items() ] ))

# write to csv
df.to_csv("/data/keeling/a/adammb4/heatwaves-physics/data/oct13_simulated_time_mois_precip_10e3events.csv")